In [ ]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

user_question = "I just discovered the course. Can I still join it?"


In [ ]:
user_question_embedding = embedding_model.encode(user_question)
user_question_embedding[0]

In [ ]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [ ]:
filter_document = []

for doc in documents:
    if doc['course'] == 'machine-learning-zoomcamp':
        filter_document.append(doc)

In [ ]:
len(filter_document)

In [75]:
from tqdm.auto import tqdm

embeddings = []
for doc in tqdm(filter_document):
    question = doc['question']
    text = doc['text']
    
    qa_text = f'{question} {text}'
    qa_text_embeddings = embedding_model.encode(qa_text)
    embeddings.append(qa_text_embeddings)

100%|██████████| 375/375 [01:05<00:00,  5.75it/s]


In [72]:

for doc in tqdm(filter_document):
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'

    doc['question_vector'] =embedding_model.encode(question)
    doc['text_vector'] =embedding_model.encode(text)
    doc['question_text_vector'] = embedding_model.encode(qa_text)



100%|██████████| 375/375 [02:22<00:00,  2.64it/s]


375

In [122]:
import numpy as np
X = np.array(embeddings)
(X.shape)


(375, 768)

In [123]:
user_question_embedding.dot(user_question_embedding)

0.9999999

In [124]:
scores = X.dot(user_question_embedding)
max(scores)

0.6506573

In [128]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filter_document, embeddings=X)
user = search_engine.search(user_question_embedding, num_results=5)

In [126]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

# df_ground_truth = pd.read_csv('ground_truth_data.csv')
# ground_truth = df_ground_truth.to_dict(orient='records')

In [134]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    question_embeddings = embedding_model.encode(q['question'])
    results = search_engine.search(question_embeddings, num_results=5)

    relevance = [d['id'] in q['document'] for d in results]
    relevance_total.append(relevance)
    

100%|██████████| 1830/1830 [01:32<00:00, 19.78it/s]


In [135]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [136]:
hit_rate(relevance_total)

0.9398907103825137

In [137]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [138]:
for doc in tqdm(filter_document):
    es_client.index(index=index_name, document=doc)

100%|██████████| 375/375 [00:08<00:00, 44.86it/s]


In [140]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_score'])

    return result_docs

In [141]:
elastic_search_knn('question_text_vector',user_question_embedding,'machine-learning-zoomcamp')

[0.82532895, 0.73585373, 0.7295, 0.72849524, 0.7252791]

In [116]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = embedding_model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)

In [117]:
relevance_total_elastic = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = question_vector_knn(q)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total_elastic.append(relevance)


100%|██████████| 1830/1830 [01:34<00:00, 19.32it/s]


In [118]:
hit_rate(relevance_total_elastic)

0.9398907103825137